In [ ]:
# default_exp sgd_shapley

In [ ]:
#export
# Copyright (c) 2020 Simon Grah <simon.grah@thalesgroup.com>
#                    Vincent Thouvenot <vincent.thouvenot@thalesgroup.com>

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

In [ ]:
#export
import numpy as np
import pandas as pd
import operator as op
from functools import reduce
import random
from tqdm import tqdm

# Projected Stochastic Gradient Shapley

> Estimate the Shapley Values using a Projected Stochastic Gradient algorithm.

## Theory

### Shapley Value definition

In Collaborative Game Theory, Shapley Values ([Shapley,1953]) can distribute a reward among players in a fairly way according to their contribution to the win in a cooperative game. We note $\mathcal{M}$ a set of $d$ players. Moreover, $v : P(\mathcal{M}) \rightarrow R_v$ a reward function such that $v(\emptyset) = 0$. The range $R_v$ can be $\Re$ or a subset of $\Re$. $P(\mathcal{M})$ is a family of sets over $\mathcal{M}$. If $S \subset \mathcal{M}\text{, } v(S)$ is the amount of wealth produced by coalition $S$ when they cooperate.

The Shapley Value of a player $j$ is a fair share of the global wealth $v(\mathcal{M})$ produced by all players together:

$$\phi_j(\mathcal{M},v) = \sum_{S \subset \mathcal{M}\backslash \{j\}}\frac{(d -|S| - 1)!|S|!}{d!}\left(v(S\cup \{j\}) - v(S)\right),$$

with $|S| = \text{cardinal}(S)$, i.e. the number of players in coalition $S$.

### Shapley Values as contrastive local attribute importance in Machine Learning

Let be $X^*\subset\Re^d$ a dataset of individuals where a Machine Learning model $f$ is trained and/or tested and $d$  the dimension of $X^*$. $d>1$ else we do not need to compute Shapley Value. We consider the attribute importance of an individual $\mathbf{x^*} = \{x_1^*, \dots, x_d^*\} \in X^*$ according to a given reference $\mathbf{r} = \{r_1, \dots, r_d\}\in X^*$.  We're looking for $\boldsymbol{\phi}=(\phi_j)_{j\in\{1, \dots, d\}}\in \Re^d$ such that:
$$ \sum_{j=1}^{d} \phi_j = f(\mathbf{x^*}) - f(\mathbf{r}), $$ 
where $\phi_j$ is the attribute contribution of feature indexed $j$.  We loosely identify each feature by its column number. Here the set of players $\mathcal{M}=\{1, \dots, d\}$ is the feature set.

In Machine Learning, a common choice for the reward is $ v(S) = \mathbb{E}[f(X) | X_S = \mathbf{x_S^*}]$, where $\mathbf{x_S^*}=(x_j^*)_{j\in S}$ and $X_S$ the element of $X$ for the coalition $S$. 
For any $S\subset\mathcal{M}$, let's define $ z(\mathbf{x^*},\mathbf{r},S)$ such that $z(\mathbf{x^*},\mathbf{r},\emptyset) = \mathbf{r}$, \ $z(\mathbf{x^*},\mathbf{r},\mathcal{M}) = \mathbf{x^*}$ and

$$ z(\mathbf{x^*},\mathbf{r},S) = (z_1,\dots, z_d) \text{ with } z_i =  x_i^* \text{ if } i \in S \text{ and } r_i  \text{ otherwise }$$ 

As explain in [Merrick,2019], each reference $\textbf{r}$ sets a single-game with $ v(S) = f(z(\mathbf{x^*},\mathbf{r},S)) - f(\mathbf{r}) $, $v(\emptyset) = 0 $ and $v(\mathcal{M}) = f(\mathbf{x^*}) - f(\mathbf{r}) $.

### Estimation of Shapley Values by a Projected Stochastic Gradient algorithm.

The Shapley Values are the only solution of a weighted linear regression problem with an equality constraint (see \cite{RVZ}, \cite{Lundberg} and \cite{Aas}):

$$ \underset{\boldsymbol{\phi}\in \Re^d}{\text{argmin}} \sum_{S \in \mathcal{M}, S \neq \{\emptyset, \mathcal{M}\}} w_S \ [v(S) - \sum_{j \in S} \phi_j]^2  $$ 

$$ \text{subject to} \sum_{i=j}^{d} \phi_j = v(\mathcal{M}) $$

where the weights $w_S = \dfrac{(d - 1)}{ {d\choose|S|} |S| (d - |S|)}$.

The function we want to minimize is:
$$ F(\boldsymbol{\phi}) = (X\boldsymbol{\phi} - Y)^T W (X\boldsymbol{\phi} - Y) = \dfrac{1}{n} \sum_{i=1}^{n} n w_i (y_i - \mathbf{x_i}^T \boldsymbol{\phi})^2 = \dfrac{1}{n} \sum_{i=1}^{n} g_i(\boldsymbol{\phi}), $$

$F$ is a $\mu$-strongly convex function defined on a convex set:
$$ K = \{\boldsymbol{\phi}; \sum_{j=1}^{d} \phi_j = v(\mathcal{M}) \ ; \ ||\boldsymbol{\phi}|| \le D \} = K_1 \cap K_2. $$

We denote $\boldsymbol{\phi_t}=(\phi_i^t)_{i\in\{1, \dots, d\}}$ the Shapley Values estimator at the iteration $t$. We also define $i_t\sim p$, with $p$ a discrete uniform distribution with support $\{1, \dots, n\}$, the coalition randomly draw for the iteration $t$. To find the unique minimum of $F$ on $K$, the Projected Stochastic gradient algorithm at each iteration $t$ follows the rules:

Sample a coalition:
$$ i_t \sim p$$
One step of gradient descent:
$$ \boldsymbol{\phi_t} = \text{Proj}_K (\boldsymbol{\phi_{t-1}} - \gamma_t \ (n p_{i_t})^{-1} \nabla g_{i_t}) $$
where
* $\gamma_t$ is a constant or decreasing step-size (also called the learning rate). $\forall t, \gamma_t > 0$;
* $\text{Proj}_K$ is the orthogonal projection on $K$;

### References

[Shapley,1953] _A value for n-person games_. Lloyd S Shapley. In Contributions to the Theory of Games, 2.28 (1953), pp. 307 - 317.

[Merrick,2019] _The Explanation Game: Explaining Machine Learning Models with Cooperative Game Theory_. Luke Merrick, Ankur Taly, 2019.

## Function 

__Parameters__

* `x`: pandas Series. The instance $\mathbf{x^*}$ for which we want to calculate Shapley value of each attribute,

* `fc`: python function. The reward function $v$,

* `r`: pandas Series. The reference $\mathbf{r}$. The Shapley values (attribute importance) is a contrastive explanation according to this individual,

* `n_iter`: integer. The number of iteration, 

* `step`: float. Step size for the SGD algorithm,

* `step_type`: string. Type of step-size learning rule. Options are: "constant", "sqrt", "inverse",

* `callback`: optional. An python object which can be called at each iteration to record distance to minimum for example. At each iteration, callback(Φ),

* `Φ_0`: numpy array or pandas Series, optional. Initial vector for the SGD algorithm.

__Returns__

* `Φ`: pandas Series. Shapley values of each attribute

In [ ]:
#export
def ncr(n, r):
    """
    Combinatorial computation: number of subsets of size r among n elements
    Efficient algorithm
    """
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer / denom

In [ ]:
#export
class SGDshapley():
    """
    Estimate the Shapley Values using a Projected Stochastic Gradient algorithm.
    """

    def __init__(self, d, C):
        """
        Calculate internal values for later purposes
        Those elements depend only on the number of features d

        Parameters
        ----------
        d : integer
            Dimension of the problem. The number of features
        """

        # Store in a dictionary for each size k of coalitions
        dict_ω_k = dict() # weights per size k
        dict_L_k = dict() # L-smooth constant per size k
        D = C * np.sqrt(d)
        for k in range(1, d):
            ω_k = (d - 1) / (ncr(d, k) * k * (d - k))
            L_k = ω_k * np.sqrt(k) * (np.sqrt(k) * D + C)
            dict_ω_k.update({k: ω_k})
            dict_L_k.update({k: L_k})
        # Summation of all L per coalition (closed formula)
        sum_L = np.sum([(d-1)/(np.sqrt(k)*(d-k)) * (np.sqrt(k)*D + C) for k in range(1, d)])
        # Probability distributions for sampling new instance
        # Classic SGD
        p = [ncr(d,k) for k in range(1,d)]
        p /= np.sum(p)
        # Importance Sampling proposal q
        q = np.array(list(dict_L_k.values())) * np.array(p)
        q /= np.sum(q)

        # Save internal attributes
        self.d = d
        self.n = 2**d - 2
        self.dict_ω_k = dict_ω_k
        self.dict_L_k = dict_L_k
        self.sum_L = sum_L
        self.p = p
        self.q = q

    def _F_i(self, Φ, x_i, y_i, ω_i):
        """Function value per instance i"""
        res = .5 * self.n * ω_i * (np.dot(x_i, Φ) - y_i)**2
        return res

    def _grad_F_i(self, Φ, x_i, y_i, ω_i):
        """Gradient vector per instance i"""
        res = ω_i * x_i[:,None].dot(x_i[None,:]).dot(Φ) - ω_i * y_i * x_i
        return res

    def _Π_1(self, x, b):
        """Projection Π on convex set K_1"""
        if np.abs((np.sum(x) - b)) <= 1e-6:
            return x
        else:
            return x - (np.sum(x) - b)/len(x)

    def _Π_2(self, x, D):
        """Projection Π on convex set K_2"""
        if np.linalg.norm(x) > D:
            return x * D / np.linalg.norm(x)
        else:
            return x

    def _Dykstra_proj(self, x, D, b, iter_proj=100, epsilon=1e-6):
        """
        Dykstra's algorithm to find orthogonal projection
        onto intersection of convex sets
        """
        xk = x.copy()
        d = len(x)
        pk, qk = np.zeros(d), np.zeros(d)
        for k in range(iter_proj):
            yk = self._Π_2(xk + pk, D)
            pk = xk + pk - yk
            if np.linalg.norm(self._Π_1(yk + qk, b) - xk, 2) <= epsilon:
                break
            else:
                xk = self._Π_1(yk + qk, b)
                qk = yk + qk - xk
        return xk

    def sgd(self, x, fc, r, n_iter=100, step=.1, step_type="sqrt",
            callback=None, Φ_0=False):
        """
        Stochastic gradient descent algorithm
        The game is defined for an element x, a reference r and function fc

        """

        # Get general information
        feature_names = list(x.index)
        f_x, f_r = fc(x.values), fc(r.values)
        v_M = f_x - f_r

        d = self.d
        n = 2**d - 2
        p = self.p
        dict_ω_k = self.dict_ω_k
        q = self.q
        dict_L_k = self.dict_L_k
        sum_L = self.sum_L

        # Store Shapley Values in a pandas Series
        if Φ_0:
            Φ = Φ_0.copy()
        else:
            Φ = np.zeros(d)
        Φ_storage = np.zeros((n_iter,d))

        # projection onto convex set K by using a simple algorithm
        # Φ = self._Dykstra_proj(Φ, D, v_M, iter_proj, epsilon=1e-6)
        Φ = Φ - (np.sum(Φ) - v_M) / d

        # Sample in advance coalition sizes
        list_k = np.random.choice(list(range(1, d)), size=n_iter, p=q)

        for t in tqdm(range(1, n_iter+1)):
            # build x_i
            k = list_k[t-1]
            indexes = np.random.permutation(d)[:k]
            x_i = np.zeros(d)
            x_i[indexes] = 1
            # Compute y_i
            z_S = np.array([x.values[j] if x_i[j] == 1 else r.values[j] for j in range(d)])
            f_S = fc(z_S)
            y_i = f_S - f_r
            # get weight ω_i
            ω_i = dict_ω_k[k]
            # calculate gradient
            p_i = dict_L_k[k] / sum_L
            grad_i = 1/(p_i) * self._grad_F_i(Φ, x_i, y_i, ω_i)
            # update Φ
            if step_type == "constant":
                Φ = Φ - step * grad_i
            elif step_type == "sqrt":
                Φ = Φ - (step/np.sqrt(t)) * grad_i
            elif step_type == "inverse":
                Φ = Φ - (step/(t)) * grad_i

            # projection onto convex set K
            # Φ = self._Dykstra_proj(Φ, D, v_M, iter_proj, epsilon=1e-6)
            Φ = Φ - (Φ.sum() - v_M) / d

            # update storage of Φ
            Φ_storage[t-1,:] = Φ

            if callback and (t % d == 0):
                callback(pd.Series(np.mean(Φ_storage[:t,:],axis=0), index=feature_names))

        # Average all Φ
        Φ = pd.Series(np.mean(Φ_storage,axis=0), index=feature_names)

        return Φ

In [ ]:
from nbdev.showdoc import *
show_doc(SGDshapley.sgd)

<h4 id="SGDshapley.sgd" class="doc_header"><code>SGDshapley.sgd</code><a href="__main__.py#L88" class="source_link" style="float:right">[source]</a></h4>

> <code>SGDshapley.sgd</code>(**`x`**, **`fc`**, **`r`**, **`n_iter`**=*`100`*, **`step`**=*`0.1`*, **`step_type`**=*`'sqrt'`*, **`callback`**=*`None`*, **`Φ_0`**=*`False`*)

Stochastic gradient descent algorithm
The game is defined for an element x, a reference r and function fc

## Example

We use a simulated dataset from the book _Elements of Statistical Learning_ ([hastie,2009], the Radial example). $X_1, \dots , X_{d}$ are standard independent Gaussian. The model is determined by:

$$ Y = \prod_{j=1}^{d} \rho(X_j), $$

where $\rho\text{: } t \rightarrow \sqrt{(0.5 \pi)} \exp(- t^2 /2)$. The regression function $f_{regr}$ is deterministic and simply defined by $f_r\text{: } \textbf{x} \rightarrow \prod_{j=1}^{d} \phi(x_j)$. For a reference $\mathbf{r^*}$ and a target $\mathbf{x^*}$, we define the reward function $v_r^{\mathbf{r^*}, \mathbf{x^*}}$ such as for each coalition $S$, $v_r^{\mathbf{r^*}, \mathbf{x^*}}(S) = f_{regr}(\mathbf{z}(\mathbf{x^*}, \mathbf{r^*}, S)) - f_{regr}(\mathbf{r^*}).$

 [hastie,2009] _The Elements of Statistical Learning: Data Mining, Inference, and Prediction, Second Edition_. Hastie, Trevor and Tibshirani, Robert and Friedman, Jerome. Springer Series in Statistics, 2009.
	

In [ ]:
d, n_samples = 5, 100
mu = np.zeros(d)
Sigma = np.zeros((d,d))
np.fill_diagonal(Sigma, [1] * d)
X = np.random.multivariate_normal(mean=mu, cov=Sigma, size=n_samples)
X = pd.DataFrame(X, columns=['x'+str(i) for i in range(1, d+1)])
def fc(x):
    phi_x = np.sqrt(.5 * np.pi) * np.exp(-0.5 * x ** 2)
    return np.prod(phi_x)
y = np.zeros(len(X))
for i in range(len(X)):
    y[i] = fc(X.values[i])
n = 2**d - 2
print("dimension = {0} ; nb of coalitions = {1}".format(str(d), str(n)))

dimension = 5 ; nb of coalitions = 30


In [ ]:
idx_r, idx_x = np.random.choice(np.arange(len(X)), size=2, replace=False)
r = X.iloc[idx_r,:]
x = X.iloc[idx_x,:]

In [ ]:
sgd_est = SGDshapley(d, C=y.max())
sgd_shap = sgd_est.sgd(x=x, fc=fc, r=r, n_iter=1000, step=.1, step_type="sqrt")

100%|██████████| 1000/1000 [00:00<00:00, 7662.56it/s]


In [ ]:
sgd_shap

x1    0.177265
x2   -0.012931
x3    0.084835
x4    0.044934
x5    0.066704
dtype: float64

In [ ]:
#hide
from shapkit.shapley_values import ShapleyValues

## Tests

In [ ]:
r_pred = fc(r.values)
x_pred = fc(x.values)
v_M = x_pred - r_pred

In [ ]:
assert np.abs(sgd_shap.sum() - v_M) <= 1e-10 

In [ ]:
true_shap = ShapleyValues(x=x, fc=fc, ref=r)
assert np.linalg.norm(sgd_shap - true_shap, 2) <= 0.1

100%|██████████| 5/5 [00:00<00:00, 469.61it/s]


## Export-

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted index.ipynb.
Converted inspector.ipynb.
Converted monte_carlo_shapley.ipynb.
Converted sgd_shapley.ipynb.
Converted shapley_values.ipynb.
